<a href="https://colab.research.google.com/github/wiwindaaulia/seq2seq_NLP/blob/main/text_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Nama : Wiwinda Aulia

##Disini saya menggunakan dataset dari kaggle yaitu bahasa turki-inggris

In [1]:
!pip install googletrans==4.0.0-rc1
!pip install keras tensorflow
!pip install -q nltk

In [2]:
# Import library yang diperlukan
import string
import re
import numpy as np
import pandas as pd
from numpy import array, argmax, random, take
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
import shutil
import tensorflow as tf
import os
import unicodedata
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [3]:
tf.random.set_seed(123)
np.random.seed(123)

###Ini adalah kumpulan dataset turki dan enggris yang bersebelahan seperti Hi  = Merhaba

In [4]:
# Tentukan path data dan baca file dengan encoding UTF-8
data_path = '/content/tur.txt'
with open(data_path, 'r', encoding='utf-8', errors='replace') as f:
    lines = f.read()

print(lines[:500])  # Cetak sebagian data


Hi.	Merhaba.
Hi.	Selam.
Run!	Kaç!
Run!	Koş!
Run.	Kaç!
Run.	Koş!
Who?	Kim?
Fire!	Ateş!
Fire!	Yangın!
Help!	Yardım et!
Jump.	Defol.
Stop!	Dur!
Stop!	Bırak!
Wait.	Bekle.
Do it.	Onu yap.
Go on.	Devam et.
Hello!	Merhaba.
Hello!	Selam.
Hurry!	Acele et!
I ran.	Koştum.
I see.	Anlıyorum.
I see.	Görüyorum.
I won.	Kazandım.
Oh no!	Oh hayır!
Relax.	Rahatla.
Smile.	Gülümse.
Attack!	Saldır!
Attack!	Hücum!
Eat it.	Bunu ye.
Eat up.	Silip süpür.
Freeze!	Kımıldama!
Freeze!	Olduğun yerde kal!
Go now.	Şimdi git.
Go


In [5]:
# Fungsi untuk memisahkan teks menjadi baris dan melakukan proses lebih lanjut
def to_lines(text):
    senst = text.strip().split('\n')
    senst = [i.split("\t")for i in senst]
    return senst

In [6]:
tur_eng = to_lines(lines)
tur_eng[:3]

[['Hi.', 'Merhaba.'], ['Hi.', 'Selam.'], ['Run!', 'Kaç!']]

#Data Cleancing

In [7]:
import re

# Bersihkan semua kolom dalam tur_eng
for row in tur_eng:
    row[:] = [re.sub(r'[^\w\s]', '', col.lower()) for col in row]

# Cek hasil
print(tur_eng[:3])


[['hi', 'merhaba'], ['hi', 'selam'], ['run', 'kaç']]


#Text to Sequence Number

In [8]:
def preprocess_sentence(sent):
    # Normalize the input sentence to decompose accented characters into their base form
    sent = "".join([c for c in unicodedata.normalize("NFD", sent) if unicodedata.category(c) != "Mn"])

    # Add space before punctuation marks like '.', '!', or '?' to tokenize them separately
    sent = re.sub(r"([!.?])", r" \1", sent)

    # Replace any sequence of characters that are not letters or punctuation with a single space
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

    # Replace multiple consecutive spaces with a single space
    sent = re.sub(r"\s+", " ", sent)

    # Convert the sentence to lowercase to ensure consistency in case
    sent = sent.lower()
    return sent

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Fungsi untuk melakukan tokenisasi pada teks
def tokenization(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# Ekstrak kolom pertama dengan list comprehension
eng_lines = [row[0] for row in tur_eng]

# Melakukan tokenisasi pada kolom pertama
eng_tokenizer = tokenization(eng_lines)

# Cek hasil tokenisasi
print(eng_tokenizer.word_index)  # Menampilkan indeks kata


{'tom': 1, 'to': 2, 'the': 3, 'i': 4, 'you': 5, 'a': 6, 'that': 7, 'is': 8, 'do': 9, 'in': 10, 'he': 11, 'mary': 12, 'of': 13, 'was': 14, 'have': 15, 'for': 16, 'me': 17, 'it': 18, 'this': 19, 'be': 20, 'what': 21, 'and': 22, 'we': 23, 'dont': 24, 'his': 25, 'with': 26, 'my': 27, 'on': 28, 'are': 29, 'know': 30, 'im': 31, 'want': 32, 'not': 33, 'think': 34, 'didnt': 35, 'at': 36, 'like': 37, 'your': 38, 'going': 39, 'about': 40, 'how': 41, 'can': 42, 'has': 43, 'go': 44, 'she': 45, 'were': 46, 'as': 47, 'did': 48, 'time': 49, 'had': 50, 'here': 51, 'will': 52, 'him': 53, 'if': 54, 'would': 55, 'get': 56, 'there': 57, 'said': 58, 'all': 59, 'been': 60, 'out': 61, 'very': 62, 'tell': 63, 'one': 64, 'when': 65, 'doesnt': 66, 'its': 67, 'told': 68, 'why': 69, 'up': 70, 'from': 71, 'her': 72, 'they': 73, 'boston': 74, 'who': 75, 'just': 76, 'us': 77, 'really': 78, 'by': 79, 'help': 80, 'should': 81, 'cant': 82, 'never': 83, 'need': 84, 'could': 85, 'ive': 86, 'good': 87, 'an': 88, 'no': 89,

In [10]:
# Menampilkan 5 kata pertama beserta indeksnya
for i in range(5):
    print(list(eng_tokenizer.word_index.keys())[i], ":", list(eng_tokenizer.word_index.values())[i])

tom : 1
to : 2
the : 3
i : 4
you : 5


In [12]:
# Menghitung ukuran kosakata bahasa Inggris
eng_vocab_size = len(eng_tokenizer.word_index) + 1
print("English Vocabulary Size: %d" % eng_vocab_size)

English Vocabulary Size: 21715


In [11]:
# Ekstrak kolom pertama dengan list comprehension
tur_lines = [row[0] for row in tur_eng]  # Perbaiki akses kolom pertama

# Tokenisasi untuk bahasa Turki
tur_tokenizer = tokenization(tur_lines)

# Mendapatkan jumlah kata unik dalam vocabulary
tur_vocab_size = len(tur_tokenizer.word_index) + 1  # Tambah 1 untuk padding
print("Turki Vocabulary Size: %d" % tur_vocab_size)

Turki Vocabulary Size: 21715


In [13]:
# Fungsi untuk mengkodekan urutan
def encode_sequences(tokenizer, length, lines):
    seq = tokenizer.texts_to_sequences(lines)
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

In [14]:
# Membagi data menjadi train dan test
from sklearn.model_selection import train_test_split
train, test = train_test_split(tur_eng, test_size=0.2, random_state=10)



In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Fungsi untuk melakukan tokenisasi pada teks
def tokenization(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# Fungsi untuk mengonversi teks menjadi urutan angka berdasarkan tokenizer
def encode_sequences(tokenizer, length, lines):
    # Konversi teks ke urutan angka
    seq = tokenizer.texts_to_sequences(lines)
    # Padding agar semua memiliki panjang yang sama
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

# Contoh dataset train (pastikan ini berbentuk list of lists)
train = [
    ['merhaba', 'hi'],
    ['nasılsın', 'how are you'],
    ['teşekkür ederim', 'thank you']
]

# Mendapatkan panjang maksimum urutan untuk padding
tur_length = max(len(row[0].split()) for row in train)  # Panjang maksimum bahasa Turki
eng_length = max(len(row[1].split()) for row in train)  # Panjang maksimum bahasa Inggris

# Tokenisasi untuk bahasa Turki dan Inggris
tur_tokenizer = tokenization([row[0] for row in train])  # Tokenizer untuk bahasa Turki
eng_tokenizer = tokenization([row[1] for row in train])  # Tokenizer untuk bahasa Inggris

# Mengonversi teks ke dalam bentuk urutan angka
train_x = encode_sequences(tur_tokenizer, tur_length, [row[0] for row in train])  # Bahasa Turki
train_y = encode_sequences(eng_tokenizer, eng_length, [row[1] for row in train])  # Bahasa Inggris


# Output hasil encoding
print("Encoded train_x:\n", train_x)
print("Encoded train_y:\n", train_y)
print("Shape of train_x:", train_x.shape)
print("Shape of train_y:", train_y.shape)


Encoded train_x:
 [[1 0]
 [2 0]
 [3 4]]
Encoded train_y:
 [[2 0 0]
 [3 4 1]
 [5 1 0]]
Shape of train_x: (3, 2)
Shape of train_y: (3, 3)


#Seq2seq

In [16]:
# === 4. Build Seq2Seq Model ===
embedding_dim = 256
units = 512
vocab_size_tur = len(tur_tokenizer.word_index) + 1
vocab_size_eng = len(eng_tokenizer.word_index) + 1

# Encoder
encoder_inputs = tf.keras.layers.Input(shape=(tur_length,))
encoder_embedding = tf.keras.layers.Embedding(vocab_size_tur, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = tf.keras.layers.LSTM(units, return_state=True)(encoder_embedding)

# Decoder
decoder_inputs = tf.keras.layers.Input(shape=(eng_length,))
decoder_embedding = tf.keras.layers.Embedding(vocab_size_eng, embedding_dim)(decoder_inputs)
decoder_lstm, _, _ = tf.keras.layers.LSTM(units, return_sequences=True, return_state=True)(
    decoder_embedding, initial_state=[state_h, state_c]
)
decoder_outputs = tf.keras.layers.Dense(vocab_size_eng, activation="softmax")(decoder_lstm)

# Build Model
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 2)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 2, 256)         │          1,280 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 3, 256)         │          1,536 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 512), (None,   │      1,574,912 │ embedding[0][0]        │
│                           │ 512), (None, 512)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 3, 512),       │      1,574,912 │ embedding_1[0][0],     │
│                           │ (None, 512), (None,    │                │ lstm[0][1], lstm[0][2] │
│                           │ 512)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 3, 6)           │          3,078 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,155,718 (12.04 MB)

 Trainable params: 3,155,718 (12.04 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
import numpy as np

# Konversi train_y menjadi array NumPy sebelum pergeseran
train_y = np.array(train_y)

# Membuat input untuk model Seq2Seq
train_input = train_y[:, :-1]
train_shifted = train_y[:, 1:]

# Padding untuk memastikan panjangnya sama
train_input = pad_sequences(train_input, maxlen=eng_length, padding="post")
train_shifted = pad_sequences(train_shifted, maxlen=eng_length, padding="post")

# Latih model
model.fit(
    [train_x, train_input],
    train_shifted,
    batch_size=64,
    epochs=10,
    validation_split=0.2
)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.1667 - loss: 1.7905 - val_accuracy: 0.6667 - val_loss: 1.7664
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6667 - loss: 1.7186 - val_accuracy: 0.6667 - val_loss: 1.7403
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step - accuracy: 0.6667 - loss: 1.6411 - val_accuracy: 0.6667 - val_loss: 1.7090
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.6667 - loss: 1.5479 - val_accuracy: 0.6667 - val_loss: 1.6697
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step - accuracy: 0.6667 - loss: 1.4304 - val_accuracy: 0.6667 - val_loss: 1.6200
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step - accuracy: 0.6667 - loss: 1.2831 - val_accuracy: 0.6667 - val_loss: 1.5574
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step - accuracy: 0.6667 - loss: 1.1094 - val_accuracy: 0.6667 - val_loss: 1.4795
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.6667 - loss: 0.9318 - val_accuracy: 0.6667 - val_loss: 1.385

In [18]:
# Buat ulang layer untuk decoder
decoder_embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size_eng, output_dim=embedding_dim)
decoder_lstm_layer = tf.keras.layers.LSTM(units, return_sequences=True, return_state=True)
decoder_dense_layer = tf.keras.layers.Dense(vocab_size_eng, activation="softmax")

In [19]:
# Input untuk decoder saat inferensi
decoder_state_input_h = tf.keras.layers.Input(shape=(units,))
decoder_state_input_c = tf.keras.layers.Input(shape=(units,))
decoder_inputs_single = tf.keras.layers.Input(shape=(1,))

# Embedding layer untuk token input decoder
decoder_embedded = decoder_embedding_layer(decoder_inputs_single)

# Gunakan LSTM decoder yang baru dideklarasikan
decoder_lstm_output, state_h2, state_c2 = decoder_lstm_layer(
    decoder_embedded, initial_state=[decoder_state_input_h, decoder_state_input_c]
)

# Gunakan dense layer yang baru dideklarasikan
decoder_outputs = decoder_dense_layer(decoder_lstm_output)

# Buat model decoder untuk inferensi
decoder_model = tf.keras.models.Model(
    [decoder_inputs_single, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs, state_h2, state_c2]
)

In [20]:
# Cek model decoder
decoder_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 1, 256)         │          1,536 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_2             │ (None, 512)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, 512)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ [(None, 1, 512),       │      1,574,912 │ embedding_2[0][0],     │
│                           │ (None, 512), (None,    │                │ input_layer_2[0][0],   │
│                           │ 512)]                  │                │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1, 6)           │          3,078 │ lstm_2[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,579,526 (6.03 MB)

 Trainable params: 1,579,526 (6.03 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
def translate(input_text):
    input_sequence = tur_tokenizer.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=tur_length, padding="post")

    encoder_states = encoder_model.predict(input_sequence)
    start_token = eng_tokenizer.word_index.get("<start>", 1)
    target_sequence = tf.constant([[start_token]])

    stop_condition = False
    decoded_sentence = []
    state_h, state_c = encoder_states[1], encoder_states[2]

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_sequence, state_h, state_c])

        # Berikan penalti untuk kata-kata yang sudah muncul
        output_tokens[0, -1, :] /= (1 + np.isin(np.arange(len(output_tokens[0, -1, :])), [eng_tokenizer.word_index.get(word, 0) for word in decoded_sentence]))

        sampled_token_index = tf.argmax(output_tokens[0, -1, :]).numpy()
        sampled_word = eng_tokenizer.index_word.get(sampled_token_index, "")

        if sampled_word == "<end>" or len(decoded_sentence) > eng_length:
            stop_condition = True
        else:
            decoded_sentence.append(sampled_word)

        target_sequence = tf.constant([[sampled_token_index]])
        state_h, state_c = h, c

    return " ".join(decoded_sentence)


In [22]:
!pip install --upgrade transformers

In [23]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Found existing installation: torch 2.6.0+cpu
Uninstalling torch-2.6.0+cpu:
  Successfully uninstalled torch-2.6.0+cpu
Found existing installation: torchvision 0.21.0+cpu
Uninstalling torchvision-0.21.0+cpu:
  Successfully uninstalled torchvision-0.21.0+cpu
Found existing installation: torchaudio 2.6.0+cpu
Uninstalling torchaudio-2.6.0+cpu:
  Successfully uninstalled torchaudio-2.6.0+cpu
Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.6.0%2Bcpu-cp311-cp311-linux_x86_64.whl.metadata (26 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.21.0%2Bcpu-cp311-cp311-linux_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.6.0%2Bcpu-cp311-cp311-linux_x86_64.whl.metadata (6.6 kB)
Using cached https://download.pytorch.org/whl/cpu/torch-2.6.0%2Bcpu-cp311-cp311-linux_x86_64.whl (178.7 MB)
Using cached https://download.pytorch.org/whl/cpu/torchvision-0.21.0%2Bcpu-cp311-cp

In [34]:
from transformers import MarianMTModel, MarianTokenizer

# Load model yang benar
model_name = "Helsinki-NLP/opus-mt-tr-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Fungsi terjemahan
def translate_with_pretrained(sentence):
    # Tambahkan token bahasa sumber di awal
    sentence = f">>tr<< {sentence}"
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)


In [36]:
# Uji coba dengan "merhaba"
input_word = "merhaba"
output_word = translate_with_pretrained(input_word)
print(output_word)

Hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello.


In [33]:
# Kalimat dalam bahasa Turki
input_sentence = (
    "Eğitim, parlak bir geleceğin kapılarını açmanın anahtarıdır. "
    "Eğitimle birlikte, bir kişi yaşam kalitesini artırabilir ve topluma olumlu katkılar sağlayabilir."
)

# Terjemahkan ke bahasa Inggris
output_sentence = translate_with_pretrained(input_sentence)
print("English:", output_sentence)

English: Education is the key to opening the door to a bright future, and with education, a person can improve the quality of life and contribute positively to society.
